<a href="https://colab.research.google.com/github/jaeohshin/ML_with_Pytorch_Sklearn_rasbt/blob/main/Car_Mileage_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd

from IPython.display import Image

In [2]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'


In [32]:
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight',
                'Acceleration', 'Model Year', 'Origin']
df = pd.read_csv(url, names=column_names,
                 na_values = "?", comment='\t',
                 sep=" ", skipinitialspace=True)

df.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
393,27.0,4,140.0,86.0,2790.0,15.6,82,1
394,44.0,4,97.0,52.0,2130.0,24.6,82,2
395,32.0,4,135.0,84.0,2295.0,11.6,82,1
396,28.0,4,120.0,79.0,2625.0,18.6,82,1
397,31.0,4,119.0,82.0,2720.0,19.4,82,1


In [33]:
print(df.isna().sum())

df = df.dropna()
df = df.reset_index(drop=True)
df.tail()

MPG             0
Cylinders       0
Displacement    0
Horsepower      6
Weight          0
Acceleration    0
Model Year      0
Origin          0
dtype: int64


,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
387,27.0,4,140.0,86.0,2790.0,15.6,82,1
388,44.0,4,97.0,52.0,2130.0,24.6,82,2
389,32.0,4,135.0,84.0,2295.0,11.6,82,1
390,28.0,4,120.0,79.0,2625.0,18.6,82,1
391,31.0,4,119.0,82.0,2720.0,19.4,82,1


In [34]:
print(df.isna().sum())

MPG             0
Cylinders       0
Displacement    0
Horsepower      0
Weight          0
Acceleration    0
Model Year      0
Origin          0
dtype: int64


In [35]:
import sklearn
import sklearn.model_selection

In [36]:
df_train, df_test = sklearn.model_selection.train_test_split(df, train_size=0.7, random_state=1)
train_stats = df_train.describe().transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
MPG,274.0,23.413869,7.649130,9.0,17.5,23.0,28.775,46.6
Cylinders,274.0,5.383212,1.706229,3.0,4.0,4.0,7.500,8.0
Displacement,274.0,188.994526,102.744326,68.0,105.0,140.0,259.500,455.0
Horsepower,274.0,103.010949,37.873946,46.0,75.0,92.5,123.750,230.0
Weight,274.0,2956.536496,846.378383,1613.0,2220.0,2750.0,3568.250,5140.0
Acceleration,274.0,15.694891,2.764859,8.5,14.0,15.5,17.300,24.8
Model Year,274.0,75.857664,3.695086,70.0,73.0,76.0,79.000,82.0
Origin,274.0,1.591241,0.803424,1.0,1.0,1.0,2.000,3.0


In [37]:
numeric_column_names = [
    'Cylinders', 'Displacement',
    'Horsepower', 'Weight',
    'Acceleration'
]

In [38]:
df_train_norm, df_test_norm = df_train.copy(), df_test.copy()
for col_name in numeric_column_names:
    mean = train_stats.loc[col_name]['mean']
    std = train_stats.loc[col_name]['std']
    df_train_norm.loc[:,col_name] = (df_train_norm.loc[:,col_name] - mean) / std
    df_test_norm.loc[:,col_name] = (df_test_norm.loc[:,col_name] - mean) / std

df_train_norm.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
203,28.0,-0.810683,-0.895373,-0.739584,-0.947019,0.255026,76,3
255,19.4,0.361492,0.418568,-0.343533,0.299468,0.544371,78,1
72,13.0,1.533667,1.148535,0.712602,1.348644,-0.613012,72,1
235,30.5,-0.810683,-0.885640,-1.056424,-1.069896,0.472035,77,1
37,14.0,1.533667,1.567050,1.636720,1.479791,-1.336376,71,1


In [39]:
boundaries = torch.tensor([73, 76, 79])

v = torch.tensor(df_train_norm['Model Year'].values)
#print(v)
df_train_norm['Model Year Bucketed'] = torch.bucketize(v, boundaries, right=True)

v = torch.tensor(df_test_norm['Model Year'].values)
df_test_norm['Model Year Bucketed'] = torch.bucketize(v, boundaries, right=True)

numeric_column_names.append('Model Year Bucketed')
df_train_norm.head()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin,Model Year Bucketed
332,35.0,-0.810683,-0.652051,-0.396340,-0.539400,-0.215161,80,2,3
391,31.0,-0.810683,-0.681250,-0.554760,-0.279469,1.340072,82,1,3
388,44.0,-0.810683,-0.895373,-1.346861,-0.976557,3.220819,82,2,3
29,27.0,-0.810683,-0.895373,-0.396340,-0.976557,-0.432171,71,3,0
273,17.0,0.361492,-0.253002,0.580585,0.216763,-0.757685,78,2,2


In [40]:
from torch.nn.functional import one_hot

total_origin = len(set(df_train_norm['Origin']))
print(total_origin)

origin_encoded = one_hot(torch.from_numpy(df_train_norm['Origin'].values) % total_origin)
print(origin_encoded.shape)

x_train_numeric = torch.tensor(df_train_norm[numeric_column_names].values)
x_train = torch.cat([x_train_numeric, origin_encoded], 1).float()

origin_encoded = one_hot(torch.from_numpy(df_test_norm['Origin'].values) % total_origin)
x_test_numeric = torch.tensor(df_test_norm[numeric_column_names].values)
x_test = torch.cat([x_test_numeric, origin_encoded], 1).float()

3


In [41]:
y_train = torch.tensor(df_train_norm['MPG'].values).float()
y_test = torch.tensor(df_test_norm['MPG'].values).float()


In [42]:
print(x_train.shape[1])

9


In [44]:
x_train[0:10,:]

tensor([[-0.8107, -0.6521, -0.3963, -0.5394, -0.2152,  3.0000,  0.0000,  0.0000,
          1.0000],
        [-0.8107, -0.6812, -0.5548, -0.2795,  1.3401,  3.0000,  0.0000,  1.0000,
          0.0000],
        [-0.8107, -0.8954, -1.3469, -0.9766,  3.2208,  3.0000,  0.0000,  0.0000,
          1.0000],
        [-0.8107, -0.8954, -0.3963, -0.9766, -0.4322,  0.0000,  1.0000,  0.0000,
          0.0000],
        [ 0.3615, -0.2530,  0.5806,  0.2168, -0.7577,  2.0000,  0.0000,  0.0000,
          1.0000],
        [-0.8107, -1.0024, -1.0036, -1.1597, -0.1790,  3.0000,  1.0000,  0.0000,
          0.0000],
        [ 1.5337,  1.1193,  1.2407,  0.8453, -1.5172,  1.0000,  0.0000,  1.0000,
          0.0000],
        [-0.8107, -0.8954, -1.2941, -0.8301,  2.8230,  0.0000,  0.0000,  0.0000,
          1.0000],
        [-0.8107, -0.4671, -0.6076,  0.3231,  1.7018,  3.0000,  0.0000,  0.0000,
          1.0000],
        [ 1.5337,  1.5768,  1.2935,  1.4869, -1.0470,  2.0000,  0.0000,  1.0000,
          0.0000]])

In [45]:
from torch.utils.data import DataLoader, TensorDataset

train_ds = TensorDataset(x_train, y_train)
batch_size = 8
torch.manual_seed(1)
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [46]:
hidden_units = [8, 4, 4]
input_size = x_train.shape[1]

all_layers= []
for hidden_unit in hidden_units:
    layer = nn.Linear(input_size, hidden_unit)
    all_layers.append(layer)
    all_layers.append(nn.ReLU())
    input_size = hidden_unit

all_layers.append(nn.Linear(hidden_units[-1], 1))

model = nn.Sequential(*all_layers)

model

Sequential(
  (0): Linear(in_features=9, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=4, bias=True)
  (3): ReLU()
  (4): Linear(in_features=4, out_features=4, bias=True)
  (5): ReLU()
  (6): Linear(in_features=4, out_features=1, bias=True)
)

In [47]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.001)

In [48]:
torch.manual_seed(1)
num_epochs = 200
log_epochs = 20

for epoch in range(num_epochs):
    loss_hist_train = 0
    for x_batch, y_batch in train_dl:
        pred = model(x_batch)[:, 0]
        loss = loss_fn(pred, y_batch)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        loss_hist_train += loss.item()
    if epoch % log_epochs == 0:
        print(f'Epoch {epoch} Loss {loss_hist_train/ len(train_dl):.4f}')

Epoch 0 Loss 516.5619
Epoch 20 Loss 8.9376
Epoch 40 Loss 7.8979
Epoch 60 Loss 7.8081
Epoch 80 Loss 8.9497
Epoch 100 Loss 6.7317
Epoch 120 Loss 6.4899
Epoch 140 Loss 6.9337
Epoch 160 Loss 6.3436
Epoch 180 Loss 5.9941


In [49]:
with torch.no_grad():
    pred = model(x_test.float())[:,0]
    loss = loss_fn(pred, y_test)
    print(f'Test MSE: {loss.item():.4f}')
    print(f'Test MAE: {nn.L1Loss()(pred, y_test).item():.4f}')

Test MSE: 8.4859
Test MAE: 2.0308
